In [1]:
from typing import Self, Optional
from collections import deque
import uuid

from qdrant_client import QdrantClient
from qdrant_client.models import (
    VectorParams, 
    Distance, 
    PointStruct,
    Batch
)

import openai

import prelude
from mychat.schema import (
    ChatMessage,
    ChatRole,
    SystemMessage,
    UserMessage,
    AssistantMessage
)
from mychat.utils import call_openai_chat

In [2]:
USER_NAME = "Isaac"
ASSISTANT_NAME = "Natia"

PROFILE = SystemMessage(
    (
        "Your name is {assistant_name}. "
        "You are a professional user growth manager in the field of GameFi. "
        "My name is {user_name}. "
        "Now, your job is to answer my questions and "
        "fulfill my request "
        "through chatting."
    ).format(
        user_name=USER_NAME,
        assistant_name=ASSISTANT_NAME
    )
)

PROFILE

[system] Your name is Natia. You are a professional user growth manager in the field of GameFi. My name is Isaac. Now, your job is to answer my questions and fulfill my request through chatting.

In [3]:
def print_message(
        message: ChatMessage, 
        do_print: bool = True
    ) -> Optional[str]:
    
    if message.role == ChatRole.System:
        message_str = message.content
        
    else:
        match message.role:
            case ChatRole.User:
                role_name = USER_NAME
            case ChatRole.Assistant:
                role_name = ASSISTANT_NAME
                
        message_str = f"[{role_name}] {message.content}"
    
    if do_print:
        print(message_str)
        return
    
    return message_str

def print_messages(
        messages: list[ChatMessage], 
        do_print: bool = True
    ) -> Optional[str]:
    
    message_str_list = []
    for message in messages:
        message_str = print_message(message, do_print=False)
        message_str_list.append(message_str)
        
    s = "\n\n".join(message_str_list)
        
    if do_print:
        print(s)
        return 
    
    return s
    
print_messages([
    PROFILE,
    UserMessage("Hello?"),
    AssistantMessage("How may I help you?")
])

Your name is Natia. You are a professional user growth manager in the field of GameFi. My name is Isaac. Now, your job is to answer my questions and fulfill my request through chatting.

[Isaac] Hello?

[Natia] How may I help you?


In [4]:
def summarize_chat_history(messages: list[ChatMessage]) -> str:
    
    chat_history = print_messages(messages, do_print=False)
    
    prompt = (
        "{profile}"
        "The following are your chat history with me: "
        "{chat_history}"
        "Summarize the above chat history by "
        "extracting the key information. "
        "Your summary is:"
    ).format(
        profile=PROFILE,
        chat_history=chat_history
    )
    
    response = call_openai_chat(
        messages=[
            UserMessage(prompt)
        ]
    )
    
    return response

summary = summarize_chat_history([
    UserMessage("Hello?"),
    AssistantMessage("How may I help you?")
])

summary

'Isaac contacted Natia, a professional user growth manager in the field of GameFi. Isaac initiated the conversation by saying hello and Natia asked how she could assist him.'

## Qdrant

In [5]:
def encode_text(text: str) -> list[float]:
    
    response = openai.Embedding.create(
        model="text-embedding-ada-002",
        input=text
    )
    
    embedding = response["data"][0]["embedding"]
    
    return embedding

In [6]:
CHAT_HISTORY_SUMMARY_COLLECTION_NAME = "chat-history-summaries"
EMBEDDING_DIM = 1536

class ChatHistoryManager(QdrantClient):
    
    def __init__(
            self,
            host: str = "localhost", 
            port: int = 6333,
            min_n_messages: int = 8,
            n_messages_to_summarize: int = 4,
            max_n_relevant_summaries: int = 5
        ):
        
        super().__init__(
            host=host,
            port=port
        )
        
        self._collection_name = CHAT_HISTORY_SUMMARY_COLLECTION_NAME
        
        self.recreate_collection(
            collection_name=self._collection_name,
            vectors_config=VectorParams(
                size=EMBEDDING_DIM,
                distance=Distance.COSINE
            )
        )
        
        self._collection = self.get_collection(self._collection_name)
        
        self._min_n_messages = min_n_messages
        self._n_messages_to_summarize = n_messages_to_summarize
        self._message_buffer: list[ChatMessage] = []
        self._max_n_relevant_summaries = max_n_relevant_summaries
    
    @property
    def collections(self) -> list[str]:
        
        return list(map(
            lambda collection: collection.name,
            self.get_collections().collections
        ))
    
    @property
    def chat_history_messages(self) -> list[ChatMessage]:
        
        return self._message_buffer
    
    def insert_message(self, message: ChatMessage):
        
        if len(self._message_buffer) >= self._min_n_messages + self._n_messages_to_summarize:
            
            messages_to_summarize = self._message_buffer[:self._n_messages_to_summarize]
            self._message_buffer = self._message_buffer[self._n_messages_to_summarize:]
            
            summary = summarize_chat_history(messages_to_summarize)
            self.insert_summary(summary)
            
        self._message_buffer.append(message)
    
    def insert_summary(self, summary: str):
        
        self.upsert(
            collection_name=self._collection_name,
            points=[
                PointStruct(
                    id=uuid.uuid1().hex,
                    payload={
                        "summary": summary
                    },
                    vector=encode_text(summary)
                )
            ]
        )
        
    def retrieve_relevant_summaries(
            self, 
            query: str,
            max_n_relevant_summaries: Optional[int] = None
        ):
        
        query_embedding = encode_text(query)
        
        if max_n_relevant_summaries is None:
            max_n_relevant_summaries = self._max_n_relevant_summaries
        
        points = self.search(
            collection_name=self._collection_name,
            query_vector=query_embedding,
            with_payload=True,
            limit=max_n_relevant_summaries
        )
        
        summaries = list(map(
            lambda point: point.payload["summary"],
            points
        ))
        
        return summaries

## Chatter

In [7]:
class Chatter:
    
    def __init__(
            self, 
            chat_history_manager: ChatHistoryManager
        ) -> None:
        
        self._chat_history_manager = chat_history_manager
        
    def __call__(self, query: str):
        
        self._chat_history_manager.insert_message(
            UserMessage(query)
        )
        
        chat_history = print_messages(
            messages=self._chat_history_manager.chat_history_messages,
            do_print=False
        )
        
        prompt = (
            "Your recent chat history with me is:"
            "{chat_history}"
            "The following summarries of previous conversations may also help:"
            "{chat_history_summary} "
            # "{query} \n"
            "Your response is:"
        ).format(
            # user_name=USER_NAME,
            chat_history=chat_history,
            chat_history_summary=print_messages(
                messages=self._chat_history_manager.retrieve_relevant_summaries(query),
                do_print=False
            ),
            query=query
        )
        
        response = call_openai_chat(
            messages=[
                PROFILE,
                UserMessage(prompt)
            ]
        )
        
        self._chat_history_manager.insert_message(
            AssistantMessage(response)
        )
        
        return response
    

In [8]:
chatter = Chatter(
    chat_history_manager=ChatHistoryManager()
)

In [9]:
chatter("Hi")

'[Assistant] Hi Isaac! How can I assist you today?'

In [10]:
chatter("What is GameFi")

'GameFi refers to the combination of blockchain technology and gaming, creating a new genre known as "play-to-earn" gaming. It allows players to earn real-world value through in-game activities, such as completing quests, trading virtual assets, or participating in tournaments. GameFi platforms often utilize non-fungible tokens (NFTs) and decentralized finance (DeFi) mechanisms to enable players to own, trade, and monetize their in-game assets. It has gained popularity due to its potential for players to earn income while enjoying their favorite games.'

In [11]:
chatter("Who are you")

'[Natia] I apologize for any confusion. I am an AI assistant designed to provide information and assistance in the field of GameFi. My purpose is to help answer your questions and fulfill your requests related to GameFi. Is there anything specific you would like to know or any assistance you need?'

In [12]:
chatter("what is my name?")

'[Natia] Your name is Isaac. Is there anything else I can assist you with, Isaac?'

In [13]:
chatter("你的项目Legend of Arcadia目前要做一系列游戏内NFT的售卖活动，你负责出一个proposal用于策划这一些活动，包括活动主题，可能的合作方，合作形式，目标客户群体；最好是crpyto nativa一些")

'[Natia] 感谢您的请求，Isaac。根据您的要求，我将为您提供一个关于Legend of Arcadia游戏内NFT售卖活动的策划提案。\n\n活动主题：探索与收集\n- 我们将以探索与收集为主题，鼓励玩家在游戏中寻找珍贵的NFT道具和角色。\n- 活动将提供多个任务和挑战，玩家需要完成这些任务来获得独特的NFT奖励。\n\n可能的合作方：\n- 艺术家和设计师：与知名艺术家和设计师合作，创作独特的NFT道具和角色，增加其稀缺性和价值。\n- NFT市场平台：与知名的NFT市场平台合作，提供一个安全可靠的交易平台，让玩家可以买卖他们的NFT道具和角色。\n- 游戏媒体和社区：与游戏媒体和社区合作，通过宣传和推广活动，吸引更多玩家参与。\n\n合作形式：\n- 艺术家和设计师合作：与艺术家和设计师签订合同，共同创作独特的NFT道具和角色。\n- NFT市场平台合作：与NFT市场平台建立合作关系，确保玩家可以安全地买卖NFT道具和角色。\n- 游戏媒体和社区合作：与游戏媒体和社区合作，通过合作推广活动，增加活动的曝光度和参与度。\n\n目标客户群体：\n- 区块链游戏爱好者：吸引那些对区块链技术和游戏相结合的概念感兴趣的玩家。\n- NFT收藏家：吸引那些对收集独特和稀缺NFT道具和角色感兴趣的玩家。\n- 游戏社区成员：吸引已经在游戏社区中活跃的玩家，增加他们对游戏的参与度。\n\n希望这个提案能够满足您的需求。如果您有任何其他问题或需要进一步的帮助，请随时告诉我。'

In [14]:
s = "I understand that you are looking for assistance in creating a proposal for a series of in-game NFT sales activities for your project, Legend of Arcadia. To plan these activities, it would be beneficial to consider the following elements:\n\n1. Activity Theme: Determine the overarching theme or concept for the NFT sales activities. This could be related to the game's storyline, characters, or unique features.\n\n2. Potential Partnerships: Identify potential partners or collaborators who can contribute to the success of the NFT sales activities. These could include other game developers, blockchain platforms, or crypto influencers.\n\n3. Collaboration Format: Define the specific ways in which you plan to collaborate with your partners. This could involve joint marketing campaigns, cross-promotions, or shared revenue models.\n\n4. Target Customer Segments: Identify the specific customer segments you want to target with your NFT sales activities. Consider factors such as demographics, gaming preferences, and crypto-native audiences.\n\n5. Crypto Integration: Explore ways to integrate crypto-native features into the NFT sales activities. This could include accepting cryptocurrency payments, leveraging blockchain technology for asset ownership, or incorporating decentralized finance (DeFi) elements.\n\nBy considering these aspects, you can create a comprehensive proposal for your in-game NFT sales activities. It's important to tailor the proposal to the unique characteristics of your project, Legend of Arcadia, and align it with your overall business goals. If you have any further questions or need more specific guidance, feel free to let me know!"

print(s)

I understand that you are looking for assistance in creating a proposal for a series of in-game NFT sales activities for your project, Legend of Arcadia. To plan these activities, it would be beneficial to consider the following elements:

1. Activity Theme: Determine the overarching theme or concept for the NFT sales activities. This could be related to the game's storyline, characters, or unique features.

2. Potential Partnerships: Identify potential partners or collaborators who can contribute to the success of the NFT sales activities. These could include other game developers, blockchain platforms, or crypto influencers.

3. Collaboration Format: Define the specific ways in which you plan to collaborate with your partners. This could involve joint marketing campaigns, cross-promotions, or shared revenue models.

4. Target Customer Segments: Identify the specific customer segments you want to target with your NFT sales activities. Consider factors such as demographics, gaming pref

In [15]:
print(chatter("Please list more details"))

[Natia] [Natia] Thank you for your request, Isaac. Based on your request, I will provide you with a proposal for a series of in-game NFT sales activities for Legend of Arcadia.

Activity Theme: Exploration and Collection
- We will focus on the theme of exploration and collection, encouraging players to search for rare NFT items and characters within the game.
- The activity will offer multiple quests and challenges, where players need to complete these tasks to earn unique NFT rewards.

Possible Collaborators:
- Artists and Designers: Collaborate with renowned artists and designers to create unique NFT items and characters, increasing their scarcity and value.
- NFT Marketplace Platforms: Partner with well-known NFT marketplace platforms to provide a secure and reliable trading platform for players to buy and sell their NFT items and characters.
- Game Media and Communities: Collaborate with game media and communities to promote and attract more players to participate through publicity